In [1]:
%load_ext autoreload
%autoreload 2

# Exercise 5

<img src="./images/05.png" width=800>

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import transforms

from torch.utils.data import Dataset, DataLoader
import numpy as np
from tqdm.autonotebook import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from utils import train_network, View, set_seed
import mlflow
from torchinfo import summary
import os
from collections import defaultdict

/tmp/ipykernel_2018/4002559147.py:10: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [ ]:
os.environ['MLFLOW_TRACKING_URI'] = './mlruns07_5'
mlflow.set_tracking_uri(os.environ.get('MLFLOW_TRACKING_URI'))

In [ ]:
mlflow.set_experiment('Exercise07_5')

<Experiment: artifact_location='/home/spakdel/my_projects/Books/Inside-Deep-Learning/Exercises_InsideDeepLearning/Chapter_07/mlruns07_1/143507330168611334', creation_time=1750415411076, experiment_id='143507330168611334', last_update_time=1750415411076, lifecycle_stage='active', name='Exercise07_1', tags={}>

In [3]:
torch.backends.cudnn.deterministic = True
set_seed(42)

In [4]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

## Dataset and DataLoader

In [ ]:
class AutoencodDataset(Dataset):
    def __init__(self, dataset):
        super().__init__()
        self.dataset = dataset
    def __len__(self):
        return len(self.dataset)
    def __getitem__(self, index):
        # x, y = self.dataset.__getitem__(index)
        x, y = self.dataset[index]
        return  x, x

In [ ]:
train_data = AutoencodDataset(torchvision.datasets.MNIST("./data", train=True, transform=transforms.ToTensor(), download=True))
test_data_xy = torchvision.datasets.MNIST("./data", train=False, transform=transforms.ToTensor(), download=True)
test_data_xx = AutoencodDataset(test_data_xy)
batch_size = 128
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data_xx, batch_size=batch_size)


## Model

In [ ]:
W = 28
H = 28
D = W * H
C =1
classes = 10

In [ ]:
def add_noise(x, device="cpu"):
    return x + torch.distributions.Normal(0, .5).sample(
        sample_shape=torch.Size(x.shape)
    ).to(device)

In [ ]:
class AdditiveGausianNoise(nn.Module):
    def __init__(self):
        super().__init__()
    
    def forward(self, x):
        if self.training:
            return add_noise(x, device=device)
        else:
            return x

### Fullt Connented Denoising Autoencoder

In [ ]:
def get_layer(in_size, out_size):
    return nn.Sequential(
        nn.Linear(in_size, out_size),
        nn.BatchNorm1d(out_size),
        nn.ReLU()
    )

In [ ]:
encoder_fully = nn.Sequential(
    nn.Flatten(),
    AdditiveGausianNoise(),
    get_layer(D, D*2),
    get_layer(D*2, D*2),
    nn.Linear(D*2, D*2),
)

decoder_fully = nn.Sequential(
    get_layer(D*2, D*2),
    get_layer(D*2, D*2),
    nn.Linear(D*2, D),
    nn.Sigmoid()
    View(-1, 1, 28, 28)
)
dn_autoencoder_fully = nn.Sequential(
    encoder_fully,
    decoder_fully,
)

### Encoder: Fully Connected, Decoder: Convolutional

In [ ]:
def conv_block(in_channels, out_channels, kernel_size=3, stride=1, padding=None):
    if padding is None:
        # Calculate padding to maintain spatial dimensions for odd kernel sizes with stride 1
        padding = (kernel_size - 1) // 2 
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding),
        nn.BatchNorm2d(out_channels),
        nn.ReLU()
    )

In [ ]:
encoder_fully = nn.Sequential(
    nn.Flatten(),
    AdditiveGausianNoise(),
    get_layer(D, D*2),
    get_layer(D*2, D*2),
    nn.Linear(D*2, D),
    View(-1, 1, 28, 28)
)

decoder_conv = nn.Sequential(
    conv_block(1, 64, kernel_size=3),
    conv_block(64, 32, kernel_size=3),
    nn.Conv2d(32, 1, kernel_size=3, stride=1, padding=1),
    nn.Sigmoid()
)

# Combined Autoencoder
dn_autoencoder_fully_conv = nn.Sequential(
    encoder_fully,
    decoder_conv,
)

### Encoder: Convolutional, Decoder: Convolutional

In [ ]:
encoder_conv = nn.Sequential(
    AdditiveGausianNoise(),
    conv_block(C, 64, kernel_size=3), 
    conv_block(64, 128, kernel_size=3),
    nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
    )

decoder_conv = nn.Sequential(
    conv_block(256, 128, kernel_size=3),
    conv_block(128, 64, kernel_size=3),
    nn.Conv2d(64, C, kernel_size=3, stride=1, padding=1),
    nn.Sigmoid() # Assuming output image pixel values are in [0, 1]
    )

# Combined Autoencoder
dn_autoencoder_conv_conv = nn.Sequential(
    encoder_conv,
    decoder_conv,
    )

## Training

In [ ]:
loss_func = nn.MSELoss()
epochs = 50
params = {
    'device': device,
    'loss_func': loss_func.__class__.__name__,
    'epochs': epochs,
    'batch_size': batch_size,
    }

In [ ]:
models = {
    'encoder:fully_decode:fully': dn_autoencoder_fully,
    'encoder:fully_decode:conv': dn_autoencoder_fully_conv,
    'encoder:conv_decode:conv': dn_autoencoder_conv_conv,
    }

In [ ]:
for experiment, model in models.items(): 
    params['experiment'] = experiment
    optimizer = optim.AdamW(model.parameters())

    with open('model_summary.txt', 'w') as f:
        f.write(str(summary(model, inpt_size=(batch_size, C, 28, 28))))
    with mlflow.start_run(nested=True, run_name=experiment):
        params['optimizer'] = optimizer.defaults
        mlflow.log_artifact('model_summary.txt')
        mlflow.log_params(params)

        results = train_network(
            model=model,
            optimizer=optimizer,
            loss_func=loss_func,
            train_loader=train_loader,
            valid_loader=test_loader,
            epochs=epochs,
            device=device,                
        )

<img src="./images/E5_train_loss.png">

<img src="./images/E5_valid_loss.png">

<img src="./images/E5_valid_loss_time.png">

#